# From Public Kernels to Bronze

In this kernel, we will share the improvement we did to public kernels, that were more than enough to reach bronze. Generous Kagglers shared some really great work, that Maxime and I had to use and improve to adapt to the consequence they had on the silver / bronze tiers. This was also the first Kaggle competition we really invested in, so we're happy with the results.

I believe we were top 70 a month ago, before $k$-opt moves were made public. Afterwards, we spent most of our time ranked between 80 and 110. We were limited by our hardware, as we were both on vacations at the end of the competition. I was running most of my work on the Kaggle kernels, which was kind of limiting (5-opt was too much...)

#### Anyways, enjoy ! 

In [ ]:
import numba
import numpy as np
import pandas as pd
from math import sqrt
from functools import lru_cache
from sklearn.neighbors import KDTree
from sympy import isprime, primerange
from tqdm import tqdm_notebook as tqdm
from ortools.constraint_solver import pywrapcp
from itertools import combinations, permutations
from scipy.spatial.distance import pdist, squareform
from ortools.constraint_solver import routing_enums_pb2
from sympy.utilities.iterables import multiset_permutations

### Loading stuff

In [ ]:
cities = pd.read_csv('../input/reeindeer/cities.csv', index_col=['CityId'])
XY = np.stack((cities.X.astype(np.float32), cities.Y.astype(np.float32)), axis=1)
is_not_prime = np.array([0 if isprime(i) else 1 for i in cities.index], dtype=np.int32)

In [ ]:
kdt = KDTree(XY)

In [ ]:
path = np.array(pd.read_csv('../input/reeindeer/path1515413.csv').Path)
# This is our current best, our method does not show improvments anymore, please use yours as input 

## First step : 2-opt
The first logical move is to use a basic 2-opt. It is quite easy to implement, but feel free to check 
> https://www.kaggle.com/kostyaatarik/close-ends-chunks-optimization-aka-2-opt 

Once this is done, you can move on to higher $k$-opt moves.

# Kostya Atarik's $k$-opt
> https://www.kaggle.com/kostyaatarik/not-a-5-and-5-halves-opt

During this competition, Kostya shared numba implementations of $k$-opt moves. His work had a huge impact on the silver/bronze tier of the leaderboard. 

### Tools

In [ ]:
@numba.jit('f8(i8, i8, i8)', nopython=True, parallel=False)
def cities_distance(offset, id_from, id_to):
    xy_from, xy_to = XY[id_from], XY[id_to]
    dx, dy = xy_from[0] - xy_to[0], xy_from[1] - xy_to[1]
    distance = sqrt(dx * dx + dy * dy)
    if offset % 10 == 9 and is_not_prime[id_from]:
        return 1.1 * distance
    return distance


@numba.jit('f8(i4, i8[:])', nopython=True, parallel=False)
def score_chunk(offset, chunk):
    pure_distance, penalty = 0.0, 0.0
    penalty_modulo = 9 - offset % 10
    for path_index in numba.prange(chunk.shape[0] - 1):
        id_from, id_to = chunk[path_index], chunk[path_index+1]
        xy_from, xy_to = XY[id_from], XY[id_to]
        dx, dy = xy_from[0] - xy_to[0], xy_from[1] - xy_to[1]
        distance = sqrt(dx * dx + dy * dy)
        pure_distance += distance
        if path_index % 10 == penalty_modulo and is_not_prime[id_from]:
            penalty += distance
    return pure_distance + 0.1 * penalty


@numba.jit('f8(i8[:])', nopython=True, parallel=False)
def score_path(path):
    return score_chunk(0, path)


@numba.jit
def chunk_scores(chunk):
    scores = np.zeros(10)
    pure_distance = 0
    for i in numba.prange(chunk.shape[0] - 1):
        id_from, id_to = chunk[i], chunk[i+1]
        xy_from, xy_to = XY[id_from], XY[id_to]
        dx, dy = xy_from[0] - xy_to[0], xy_from[1] - xy_to[1]
        distance = sqrt(dx * dx + dy * dy)
        pure_distance += distance
        if is_not_prime[id_from]:
            scores[9-i%10] += distance
    scores *= 0.1
    scores += pure_distance
    return scores


@numba.jit('f8(i8, i8, i8[:], i8[:], i8[:], i8, f8[:,:], i8[:])', nopython=True, parallel=False)
def score_compound_chunk(offset, head, firsts, lasts, lens, tail, scores, indexes):
    score = 0.0
    last_city_id = head
    for i in numba.prange(len(indexes)):
        index = indexes[i]
        first, last, chunk_len = firsts[index], lasts[index], lens[index]
        score += cities_distance(offset, last_city_id, first)
        score += scores[index, (offset + 1) % 10]
        last_city_id = last
        offset += chunk_len
    return score + cities_distance(offset, last_city_id, tail)


@numba.jit('i8(i8, i8, i8[:], i8[:], i8[:], i8, f8[:,:], i8[:,:], f8)', nopython=True, parallel=False)
def best_score_permutation_index(offset, head, firsts, lasts, lens, tail, scores, indexes, best_score):
    best_index = -1
    for i in numba.prange(len(indexes)):
        score = score_compound_chunk(offset, head, firsts, lasts, lens, tail, scores, indexes[i])
        if score < best_score:
            best_index, best_score = i, score
    return best_index


@numba.jit('f8(i8[:])', nopython=True, parallel=False)
def sum_distance(ids):
    res = 0
    for i in numba.prange(len(ids)):
        for j in numba.prange(i + 1, len(ids)):
            res += cities_distance(0, ids[i], ids[j])
    return res


@lru_cache(maxsize=None)
def indexes_permutations(n):
    return np.array(list(map(list, permutations(range(n)))))

### To the algorithm ...

The following function generates the combinations :
- n : length of the combinations (for a n-opt)
- neighbours : parameter of the "neighbour" kdt query
- radius : parameter of the "radius" kdt query 

I did not change anything here, just refactored the code.

In [ ]:
def get_uplets(n, neighbours=10, radius=10):
    uplets = set()  # To ensure uniqueness
    for i in tqdm(cities.index):
        # Query by neighbour
        dists, neibs = kdt.query([XY[i]], neighbours)
        for comb in combinations(neibs[0], n):
            if all(comb): # Removing combinations containing 0
                uplets.add(tuple(sorted(comb)))
                
        # Query by radius
        neibs = kdt.query_radius([XY[i]], radius, count_only=False, return_distance=False)
        for comb in combinations(neibs[0], n):
            if all(comb):
                uplets.add(tuple(sorted(comb)))

    print(f'{len(uplets)} cities {n}-uplets are selected.')
    
    # Sorting by distance
    uplets = np.array(list(uplets))
    distances = np.array(list(map(sum_distance, uplets)))
    order = distances.argsort()
    uplets = uplets[order]
    
    return uplets

In [ ]:
score_path(path)

## 3-opt
To improve Kostya's work, I added the possibility to reverse a chunk in the path. 
The implementation is quite naïve here.

In [ ]:
trees = get_uplets(3, 5, 5)  # Tweak those parameters

In [ ]:
path_index = np.argsort(path[:-1])
print(f'Total score is {score_path(path):.2f}.')

for ids in tqdm(trees):
    i1, i2, i3 = np.sort(path_index[ids])
    head, tail = path[i1-1], path[i3+1]
    
    # No inversion, First chunk inversed, Second chunk inversed, both chunks inversed
    all_chunks = [[path[i1:i1+1], path[i1+1:i2], path[i2:i2+1], path[i2+1:i3], path[i3:i3+1]], 
                  [path[i1:i1+1], path[i1+1:i2][::-1], path[i2:i2+1], path[i2+1:i3], path[i3:i3+1]],
                  [path[i1:i1+1], path[i1+1:i2], path[i2:i2+1], path[i2+1:i3][::-1], path[i3:i3+1]], 
                  [path[i1:i1+1], path[i1+1:i2][::-1], path[i2:i2+1], path[i2+1:i3][::-1], path[i3:i3+1]]]
    
    for i, chunks in enumerate(all_chunks):
        chunks = [chunk for chunk in chunks if len(chunk)] # Removing empty chunks
        scores = np.array([chunk_scores(chunk) for chunk in chunks]) # Scoring chunks
        lens = np.array([len(chunk) for chunk in chunks]) 
        firsts = np.array([chunk[0] for chunk in chunks])
        lasts = np.array([chunk[-1] for chunk in chunks])

        if i == 0: # Original chunk score
            best_score = score_compound_chunk(i1-1, head, firsts, lasts, lens, tail, scores, indexes_permutations(len(chunks))[0])
        
        # Best scoring permutation of the chunks
        index = best_score_permutation_index(i1-1, head, firsts, lasts, lens, tail, scores, indexes_permutations(len(chunks)), best_score)
        
        if index > 0: # If it is not the not permuted chunk
            perm = [chunks[i] for i in indexes_permutations(len(chunks))[index]]
            path[i1-1:i3+2] = np.concatenate([[head], np.concatenate(perm), [tail]]) # Applying combination
            path_index = np.argsort(path[:-1])
            print(f'New total score is {score_path(path):.3f}. Permutating path at indexes {i1}, {i2}, {i3}.')
            break # We move to the next combination
            
    break # Uncomment to run the algorithm

In [ ]:
score_path(path)

## 4-opt - Same thing, more permutations.
The 4-opt worked really well for us, and as we did not have a lot of computing ressources, it got us better improvments than the 5-opt.
Big plus is that there is almost no changes to do to our 3-opt implementation.

In [ ]:
fours = get_uplets(4, 5, 0) # Tweak this as well

In [ ]:
path_index = np.argsort(path[:-1])
print(f'Total score is {score_path(path):.2f}.')

for ids in tqdm(fours):
    i1, i2, i3, i4 = np.sort(path_index[ids])  # Added i4
    head, tail = path[i1-1], path[i4+1]  # Tail changed
    
    # New permutations 
    all_chunks = [[path[i1:i1+1], path[i1+1:i2], path[i2:i2+1], path[i2+1:i3], path[i3:i3+1], path[i3+1:i4], path[i4:i4+1]], 
                  [path[i1:i1+1], path[i1+1:i2][::-1], path[i2:i2+1], path[i2+1:i3], path[i3:i3+1], path[i3+1:i4], path[i4:i4+1]], 
                  [path[i1:i1+1], path[i1+1:i2], path[i2:i2+1], path[i2+1:i3][::-1], path[i3:i3+1], path[i3+1:i4], path[i4:i4+1]], 
                  [path[i1:i1+1], path[i1+1:i2], path[i2:i2+1], path[i2+1:i3], path[i3:i3+1], path[i3+1:i4][::-1], path[i4:i4+1]], 
                  [path[i1:i1+1], path[i1+1:i2][::-1], path[i2:i2+1], path[i2+1:i3][::-1], path[i3:i3+1], path[i3+1:i4], path[i4:i4+1]], 
                  [path[i1:i1+1], path[i1+1:i2][::-1], path[i2:i2+1], path[i2+1:i3], path[i3:i3+1], path[i3+1:i4][::-1], path[i4:i4+1]], 
                  [path[i1:i1+1], path[i1+1:i2], path[i2:i2+1], path[i2+1:i3][::-1], path[i3:i3+1], path[i3+1:i4][::-1], path[i4:i4+1]], 
                  [path[i1:i1+1], path[i1+1:i2][::-1], path[i2:i2+1], path[i2+1:i3][::-1], path[i3:i3+1], path[i3+1:i4][::-1], path[i4:i4+1]]
                 ]

    for i, chunks in enumerate(all_chunks):
        chunks = [chunk for chunk in chunks if len(chunk)]
        scores = np.array([chunk_scores(chunk) for chunk in chunks])
        lens = np.array([len(chunk) for chunk in chunks])
        firsts = np.array([chunk[0] for chunk in chunks])
        lasts = np.array([chunk[-1] for chunk in chunks])

        if i == 0:
            best_score = score_compound_chunk(i1-1, head, firsts, lasts, lens, tail, scores, indexes_permutations(len(chunks))[0])
        
        index = best_score_permutation_index(i1-1, head, firsts, lasts, lens, tail, scores, indexes_permutations(len(chunks)), best_score)
        
        if index > 0:
            perm = [chunks[i] for i in indexes_permutations(len(chunks))[index]]
            path[i1-1:i4+2] = np.concatenate([[head], np.concatenate(perm), [tail]])  #i3 -> i4
            path_index = np.argsort(path[:-1])
            print(f'New total score is {score_path(path):.3f}. Permutating path at indexes {i1}, {i2}, {i3}, {i4}.') #Added i4
            break
    break # Uncomment to run the algorithm

In [ ]:
score_path(path)

## 5-opt, 6-opt and more.
If you have enough time, doing the same thing on a 5/6 opt is easy.

# Babachan's Local Optimization
Next thing is to optimize subpaths of our tour, we used babachan's work :
> https://www.kaggle.com/hblearn/local-optimization-using-google-or-tool

The only thing we did was removing randomness and trying more subpaths.

### Tools

In [ ]:
num_iters = 300
path_df = pd.DataFrame({"Path": path})
cities2 = cities.reset_index().rename(columns={"CityId": "Path"})
path_df = path_df.merge(cities2,how='left',on='Path')
pnums = list(primerange(0, path_df.shape[0]))

status_dict = {0: 'ROUTING_NOT_SOLVED', 
               1: 'ROUTING_SUCCESS', 
               2: 'ROUTING_FAIL',
               3: 'ROUTING_FAIL_TIMEOUT',
               4: 'ROUTING_INVALID'}

def create_mat(df):
    mat = pdist(df)
    return squareform(mat)

def create_distance_callback(dist_matrix):
    def distance_callback(from_node, to_node):
        return int(dist_matrix[from_node][to_node])
    return distance_callback

def optimize(df, startnode, stopnode, fixed):     
    num_nodes = df.shape[0]
    mat = create_mat(df)
    dist_callback = create_distance_callback(mat)
    search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
    search_parameters.solution_limit = num_iters 
    search_parameters.first_solution_strategy = (
                                    routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_INSERTION)
    search_parameters.local_search_metaheuristic = (
                            routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)

    if fixed:
        routemodel = pywrapcp.RoutingModel(num_nodes, 1, [startnode], [stopnode])
    else:
        routemodel = pywrapcp.RoutingModel(num_nodes, 1, startnode)
    routemodel.SetArcCostEvaluatorOfAllVehicles(dist_callback)
    
    assignment = routemodel.SolveWithParameters(search_parameters)
    return routemodel, assignment

def get_route(df, startnode, stopnode, fixed): 
    routemodel, assignment = optimize(df, int(startnode), int(stopnode), fixed)
    route_number = 0
    node = routemodel.Start(route_number)
    route = []
    while not routemodel.IsEnd(node):
        route.append(node) 
        node = assignment.Value(routemodel.NextVar(node))
    return route

### Optimization

In [ ]:
def run_opt(df, m):
    print(f'm = {m}')
    
    # Given a length of subpath m, the subpath we optimize start at 1, m/2+1, m+1, ... , len(path) - m - 1
    # Therefore, n is :
    n = int(path_df.shape[0] / m) * 2
    
    for i in tqdm(range(n)):
        startpoint = int(1 +  i * (df.shape[0] - m - 2) / n)
        endpoint = min((startpoint + m), df.shape[0])

        district = df.iloc[startpoint:endpoint, :3].copy()
        district = district.reset_index()
        locations = district[['X', 'Y']].values

        segnodes = get_route(locations, 0, (m-1), fixed=True)
        ord_district = district.iloc[segnodes]
        segment = ord_district.index.tolist()

        temp = district.loc[segment, ['Path','X', 'Y']].reset_index()
        district_2 = district.copy()
        district_2.iloc[:(m-1),1:] = temp.copy()
        district = district.set_index('index')
        district_2 = district_2.set_index('index')

        district['step'] = np.sqrt((district.X - district.X.shift())**2 + (district.Y - district.Y.shift())**2)
        district['step_adj'] = np.where((district.index) % 10 != 0, district.step, district.step + 
                                        district.step*0.1*(~district.Path.shift().isin(pnums)))
        district_2['step'] = np.sqrt((district_2.X - district_2.X.shift())**2 + (district_2.Y - district_2.Y.shift())**2)
        district_2['step_adj'] = np.where((district_2.index) % 10 != 0, district_2.step, district_2.step + 
                                          district_2.step*0.1*(~district_2.Path.shift().isin(pnums)))

        check_dist = district.step_adj.sum() > district_2.step_adj.sum()
        if check_dist:
            df.iloc[startpoint:endpoint,0:3] = district_2
            print(f"Improved of {district.step_adj.sum() - district_2.step_adj.sum()}")
        
        break #Uncomment to run the algorithm

In [ ]:
m = 40  # 30 and 50 are also good choices
run_opt(path_df, m)
path = path_df['Path'].values

In [ ]:
score_path(path)

Once you have those two optimization methods, the idea is to combine them until you're stuck, play with parameters,  etc...

## Next step : How to reach Silver
- Higher ranked people noticed that algorithms worked better by incrementing the prime penalty (Penalizing by 1%, 2%, ..., 10% instead of 10% directly). Unfortunately, we missed this idea. 
It is quite simple to implement, but a really clever trick. 
- You could also increment the gain to avoid local minima (accept only moves that have a high enough improvement)

See : 
> https://www.kaggle.com/c/traveling-santa-2018-prime-paths/discussion/77250

## Submission

In [ ]:
def make_submission(name, path):
    pd.DataFrame({'Path': path}).to_csv(f'{name}.csv', index=False)

In [ ]:
make_submission("path" + str(int(score_path(path))), path)

## Final word : 

- Would our score have been better without the sharings of other amazing Kagglers ?  Probably not.

- Would we have been higher ranked ? Probably, yes.

But we do believe that the most important thing is that we learned a lot more with these ideas, than with our sloppy implementation of local optimization and $k$-opt moves.

So thanks again to everybody who shared anything during the competition.

#### Thanks for reading ! 